# Converting 'access.log' to tabular data

With this notebook, we will download the access.log file from our repository. From there, we will parse the data line by line and extract it into a dataframe. Pandas will let us download the df as a file, which we will name "access.tsv".

### Steps
1. Clone the repository
2. Open the file and parse it line by line
3. Extract important information from the request and create a dataset
4. Establish a dataframe with pandas
5. Download the data as "access.tsv"


In [19]:
# First, we need to clone the library so we can have access to the log file.

!git clone https://github.com/brain-image-library/py-brain-logs.git

fatal: destination path 'py-brain-logs' already exists and is not an empty directory.


In [15]:
# Ensure that 'py-brain-logs' is in our directory
!ls


py-brain-logs  sample_data


A sample line of access.log looks like this:

```51.222.253.19 - - [15/May/2022:03:30:09 -0400] "GET /56/77/567794f41ad2dccd/mouseID_394528-18867/1059286962_18867_4330-X29413-Y7746.swc HTTP/2.0" 404 146 "-" "Mozilla/5.0 (compatible; AhrefsBot/7.0; +http://ahrefs.com/robot/)" "-"```

We will split this by a space, and achieve the following list named `fields`:

```['51.222.253.19', '-', '-', '[15/May/2022:03:30:09', '-0400]', '"GET', '/56/77/567794f41ad2dccd/mouseID_394528-18867/1059286962_18867_4330-X29413-Y7746.swc', 'HTTP/2.0"', '404', '146', '"-"', '"Mozilla/5.0', '(compatible;', 'AhrefsBot/7.0;', '+http://ahrefs.com/robot/)"', '"-"\n']```

So, we can take the following data

- fields[0] as the ip address
- fields[3] and fields[4] for the date, while making it into a `dateframe` for usability.
- fields[5] for the method type of the request
- fields[6] for the url
- fields[8] for the status code of the request
- fields[9] for the size of the request

And lastly, we can check if the request is from a bot if we check if the request contains 'bot' and that the bot is not in the url.

In [20]:
import pandas as pd

from datetime import datetime

# Open the file and parse through it
with open('py-brain-logs/data/access.log', 'r') as file:
    data = []
    for line in file.readlines():
        # Split the log entry based on spaces
        fields = line.split(' ')

        # Create the following dataset based on our explanation of the fields list above.
        dataset = {
            'ip': fields[0],
            'date': datetime.strptime(fields[3].lstrip('[') + fields[4].strip(']'), '%d/%b/%Y:%H:%M:%S%z'),
            'method': fields[5].strip('').strip('"'),
            'url': fields[6],
            'status_code': fields[8],
            'size': fields[9]
        }

        dataset['is_bot'] = 'bot' in line and 'bot' not in dataset['url']

        data.append(dataset)

# Create a dataframe
df = pd.DataFrame(data)
df

,ip,date,method,url,status_code,size,is_bot
0,51.222.253.19,2022-05-15 03:30:09-04:00,GET,/56/77/567794f41ad2dccd/mouseID_394528-18867/1...,404,146,True
1,51.222.253.19,2022-05-15 03:30:17-04:00,GET,/56/77/567794f41ad2dccd/mouseID_394528-18867/1...,404,146,True
2,51.222.253.12,2022-05-15 03:30:24-04:00,GET,/56/77/567794f41ad2dccd/mouseID_394528-18867/1...,404,146,True
3,51.222.253.14,2022-05-15 03:30:33-04:00,GET,/56/77/567794f41ad2dccd/mouseID_394528-18867/1...,404,146,True
4,51.222.253.1,2022-05-15 03:30:40-04:00,GET,/56/77/567794f41ad2dccd/mouseID_394528-18867/1...,404,146,True
...,...,...,...,...,...,...,...
38505,185.191.171.17,2022-05-17 12:23:57-04:00,GET,/biccn/mueller/mouselight/2019-09-06/3/5/5/4/1/,200,1331,True
38506,185.191.171.14,2022-05-17 12:23:58-04:00,GET,/biccn/mueller/mouselight/2018-12-01/4/1/2/8/6/7/,200,479,True
38507,157.90.181.151,2022-05-17 12:23:59-04:00,GET,/bf/fb/,200,158,False
38508,185.191.171.14,2022-05-17 12:23:59-04:00,GET,/2f/27/2f27b45f2590ec86/2019-09-06/ktx/6/3/3/5/1/,200,1224,True


In [17]:
# Convert the dataframe to a tsv file and download it to the local machine.

df.to_csv('output.tsv', sep='\t', index=False)

from google.colab import files
files.download('access.tsv')